WITH LLAMA-2

In [ ]:
!pip install pinecone-client langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install pypdf
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import os
import sys
import pinecone
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain

In [ ]:
import os

# api keys
os.environ["REPLICATE_API_TOKEN"] = ""
os.environ['PINECONE_API_KEY'] = ''
os.environ['PINECONE_ENV']= 'us-east-1'

# loading pdf

loader = PyPDFLoader("LoRA_research.pdf")
documents = loader.load()

# splitting data into chunks

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# using embeddings from hugging face
embeddings = HuggingFaceEmbeddings()

# connecting to vector db
index_name = "pdfchat"
index = pinecone.Index(index_name, host="us-east-1-gcp-free.pinecone.io")
vectordb = Pinecone.from_documents(texts, embeddings, index_name=index_name)

# initialize Replicate to host on cloud
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75, "max_length": 3000}
)

# QA retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

In [ ]:
# chat w the chatbot

chat_history = []
while True:
    query = input('Prompt: ')
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt: who is the author
Answer:  Based on the information provided, the author of the article is Kailash Thiyagarajan.

Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


WITH GPT-4

In [ ]:
!pip install langchain openai PyPDF2 faiss-cpu
!pip install -U langchain-community
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling pack

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/MyDrive/"

Mounted at /content/drive


In [ ]:
reader = PdfReader("/content/drive/MyDrive/MAF-Statements-2023.pdf")

In [ ]:
reader

In [ ]:
raw_text = ""
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
raw_text

" \n   \n \n  \n \nPRESS RELEASE  \n \nMajid Al Futaim Announces Full Year 2023 Financial Results  \n \nKey Highlights:  \n• Reported a 12% increase in EBITDA to AED 4.6 billion  and a 1% increase in revenue to 34.5 \nbillion year on year.  \n• Achieved Net Profit of AED 2.7 billion , an increase of 12% year on year. \n• Majid Al Futtaim – Properties record ed a 20% year -on-year increase in revenue to AED 6.9 \nbillion  and 21% increase in EBITDA to AED 3.6 billion due to the success of UAE -based  \nshopping malls  and Tilal Al Ghaf. \n• Majid Al Futtaim – Retail digital sales revenue grew by 17 % to  AED 2.6 billion. \n• Majid Al Futtaim – Lifestyle revenue increased by 29% and crossed the AED 1 billion mark for \nthe first time . \n• Maintained its low -risk ESG rating by Sustainalytics.  \n• Retained  its BBB credit rating in latest Standard & Poor’s (S&P) and Fitch Ratings reports . \n \nDubai, United Arab Emirates, 14 March 2024 : Majid Al Futtaim  (“the Group ”), the leading sh

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function = len,
    )
texts = text_splitter.split_text(raw_text)

In [ ]:
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
docsearch

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

WITH GOOGLE T5

In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda

In [ ]:
from langchain.document_loaders import PyPDFLoader
import textwrap
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Load the PDF file
loader = PyPDFLoader("MAF-Statements-2023.pdf")
documents = loader.load()

#print(documents)

# Preprocessing
def wrap_text_preserve_newlines(text, width=100):
    # Split the text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line to the specified width
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# Text Splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(texts[0])
print(len(texts))

# Embedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
doc = FAISS.from_documents(texts, embeddings)

query = input("Enter your query: ")
docs = doc.similarity_search(query)

pdf_text = docs[0].page_content
print(wrap_text_preserve_newlines(pdf_text))

#Q-A
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})
chain = load_qa_chain(llm, chain_type="stuff")

queryNew = input("Enter your query: ")
docsNew = doc.similarity_search(queryNew)

chain.run(input_documents=docsNew, question=queryNew)

page_content='PRESS RELEASE  \n \nMajid Al Futaim Announces Full Year 2023 Financial Results  \n \nKey Highlights:  \n• Reported a 12% increase in EBITDA to AED 4.6 billion  and a 1% increase in revenue to 34.5 \nbillion year on year.  \n• Achieved Net Profit of AED 2.7 billion , an increase of 12% year on year. \n• Majid Al Futtaim – Properties record ed a 20% year -on-year increase in revenue to AED 6.9 \nbillion  and 21% increase in EBITDA to AED 3.6 billion due to the success of UAE -based  \nshopping malls  and Tilal Al Ghaf. \n• Majid Al Futtaim – Retail digital sales revenue grew by 17 % to  AED 2.6 billion. \n• Majid Al Futtaim – Lifestyle revenue increased by 29% and crossed the AED 1 billion mark for \nthe first time . \n• Maintained its low -risk ESG rating by Sustainalytics.  \n• Retained  its BBB credit rating in latest Standard & Poor’s (S&P) and Fitch Ratings reports . \n \nDubai, United Arab Emirates, 14 March 2024 : Majid Al Futtaim  (“the Group ”), the leading shoppin

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Enter your query: what are the key highlights
Operating Company Performance
 Majid Al Futtaim –  Properties: 2023  saw revenue gr ow by 20 % year -on-year to AED 6.9 billion
and
EBITDA increase  by 21%  to AED 3.6 billion . This success  was due to record breaking footfall and
higher
occupancy rates benefitting UAE -based malls as well as a solid performance from the Tilal Al Ghaf
development.
 Key business performance indicators maintained a positive trajectory, with the Shopping Malls
business witnessing record tenant sales of AED 30 billion , an increase in overall occupancy  to 96%
and
an 8% rise in footfall, welcoming 232 m visitors .

Moreover, Majid Al Futtaim Hotels witnessed  an 82% occupancy rate on  hotel rooms in 2023 . Hotels
revenues increased 4% to AED 700 million in 2023, with RevPAR growing by 5% compared to 2022.
These results were driven by the strong tourism in Dubai, with the city welcoming 15.37 million
overnight international visitors in the first 11 months of 20

BadRequestError:  (Request ID: 2QcfeAQ5pL6UDyopwR7k7)

Bad request:
Authorization header is correct, but the token seems invalid

WITH OPENAI

In [ ]:
!pip install langchain openai panel transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
# panel widget for user input
import panel as pn
pn.extension()

question_input = pn.widgets.TextInput(name='Question:')
submit_button = pn.widgets.Button(name = 'Submit', button_type = 'primary')

#Q&A

import openai
import langchain

openai.api_key = ''

#Q&A function

def answer_question(q):
    # Preprocess the text
    preprocessed_question = langchain.preprocess_text(q)

    # Use the GPT-3 model to generate the answer
    response = openai.Completion.create(
        engine='davinci',
        prompt=preprocessed_question,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Extract the answer from the response
    answer = response.choices[0].text.strip()

    return answer

output = pn.pane.Markdown()

def update_output(event):
    question = question_input.value
    answer = generate_response(question)
    output.object = f"Answer: {answer}"

submit_button.on_click(update_output)

app = pn.Column(
    question_input,
    submit_button,
    output
)

app.servable()

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 481, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 442, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 479, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 416, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/widgets/button.py", line 247, in _process_event
 self.clicks += 1
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 528, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameters.py", line 543, in __set__
 super().__set__(obj,val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1552, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2525, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2505, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-11-bb3d6dfa2e6f>", line 40, in update_output
 answer = generate_response(question)
NameError: name 'generate_response' is not defined

Column
    [0] TextInput(name='Question:')
    [1] Button(button_type='primary', name='Submit')
    [2] Markdown(None)

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.31.1
    Uninstalling openai-1.31.1:
      Successfully uninstalled openai-1.31.1


In [ ]:
import os
import panel as pn
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# Set up Panel extension
pn.extension()

# Set up OpenAI API key
openai.api_key = ''

# Load the PDF file
pdf_path = "MAF-Statements-2023.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Combine text chunks into a single string for context
pdf_text = "\n".join([text.page_content for text in texts])


# Define the function to answer the question
def generate_response(question):
    # Use OpenAI's GPT-3 to generate an answer with context from the PDF
    response = openai.Completion.create(
        engine='gpt-3.5-turbo',
        prompt=f"Context: {pdf_text}\n\nQuestion: {question}\n\nAnswer:",
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Extract the answer from the response
    answer = response.choices[0].text.strip()

    return answer

# Panel widgets for user input
question_input = pn.widgets.TextInput(name='Question:')
submit_button = pn.widgets.Button(name='Submit', button_type='primary')
output = pn.pane.Markdown()

# updating output on button click
def update_output(event):
    question = question_input.value
    answer = generate_response(question)
    output.object = f"Answer: {answer}"

submit_button.on_click(update_output)

app = pn.Column(
    question_input,
    submit_button,
    output
)

app.servable()

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 481, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 442, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 479, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 416, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/widgets/button.py", line 247, in _process_event
 self.clicks += 1
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 528, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameters.py", line 543, in __set__
 super().__set__(obj,val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1552, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2525, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2505, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-1-ac08844551c4>", line 53, in update_output
 answer = generate_response(question)
 File "<ipython-input-1-ac08844551c4>", line 31, in generate_response
 response = openai.Completion.create(
 File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/completion.py", line 25, in create
 return super().create(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
 response, _, api_key = requestor.request(
 File "/usr/local/lib/python3.10

Column
    [0] TextInput(name='Question:')
    [1] Button(button_type='primary', name='Submit')
    [2] Markdown(None)